In [3]:
import json
import mysql.connector

def parse_config_file(path='./config.json'):

    with open(path, 'r') as config_file:
        config_data = json.load(config_file)

    server = config_data['server']
    username = config_data['username']
    password = config_data['password']
    database = config_data['database']
    personnel_table_name = config_data['personnel_table_name']
    attendance_table_name = config_data['attendance_table_name']
    mysql_unknown_name_table = config_data['mysql_unknown_name_table']

    return server, username, password, database, personnel_table_name, attendance_table_name, mysql_unknown_name_table

def connect_mysql_database(server: str, username: str, password : str) -> tuple :
    '''Connects to a MySQL server and sets the table names for 3 required tables'''
    try:
        config = {
            "host": server,
            "user": username,
            "password": password
        }

        connection = mysql.connector.connect(**config)
        cursor = connection.cursor()
        
        return connection, cursor
    
    except:
        print("Couldn't connect to MySQL database server! Please check your credentials.")
        exit()

server, username, password, database, personnel_table_name, attendance_table_name, mysql_unknown_name_table = parse_config_file(path='./config.json')

connection, cursor = connect_mysql_database(server, username, password)

# Creating schema (database)
cursor.execute(f"""
CREATE DATABASE IF NOT EXISTS {database};
""")

# Using the created databse
cursor.execute(f"""
USE {database};
""")

# Creating attendance table
cursor.execute(f'''
CREATE TABLE {attendance_table_name} (
    student_id INT NOT NULL,
    university VARCHAR(255) NOT NULL,
    date DATE NULL,
    attendance BOOLEAN NULL,
    in_time TIME NULL,
    entry_exit TEXT NULL
);
''')

# Creating personnel table
cursor.execute(f'''
CREATE TABLE {personnel_table_name} (
    student_id INT NOT NULL,
    university VARCHAR(255) NOT NULL,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL
);
''')

cursor.close()
connection.close()


In [24]:
(datetime.datetime.strptime(datetime.datetime.now().strftime('%H:%M:%S'), '%H:%M:%S') - datetime.datetime.strptime('11:11:00', '%H:%M:%S')).total_seconds()

-3361.0

In [10]:
datetime.datetime.strptime('8:53:00', '%H:%M:%S')

datetime.datetime(1900, 1, 1, 8, 53)

In [3]:
import datetime
datetime.date.today()

datetime.date(2023, 9, 18)

In [1]:
from datetime import datetime, timedelta

def has_time_interval_passed(time_interval_seconds, target_time_str):
    try:
        # Convert the target time string to a datetime object
        target_time = datetime.strptime(target_time_str, '%H:%M:%S')

        # Calculate the current time
        current_time = datetime.now().time()

        # Calculate the time difference between target time and current time
        time_difference = timedelta(
            hours=target_time.hour,
            minutes=target_time.minute,
            seconds=target_time.second
        )

        # Check if the time difference has surpassed the specified interval
        return time_difference.total_seconds() >= time_interval_seconds

    except ValueError:
        # Handle invalid time string
        print("Invalid time format. Please use 'HH:MM:SS' format.")
        return False

# Example usage:
time_interval_seconds = 3600  # 1 hour
target_time_str = '9:14:02'

result = has_time_interval_passed(time_interval_seconds, target_time_str)

if result:
    print("Time interval has passed.")
else:
    print("Time interval has not passed yet.")


Time interval has passed.


In [31]:
import mysql.connector
import cv2
import pickle
import datetime
import face_recognition
import json

'''
CREATE TABLE attendance (
    student_id INT NOT NULL,
    university VARCHAR(255) NOT NULL,
    date DATE NULL,
    attendance BOOLEAN NULL,
    in_time TIME NULL,
    entry_exit TEXT NULL
);
'''

'''
CREATE TABLE personnel (
    student_id INT NOT NULL,
    university VARCHAR(255) NOT NULL,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL
);
'''


class Session:


    def __init__(self, face_location_model='hog', face_encoding_model = 'small'):

        self.face_location_model = face_location_model #'cnn' has better accuracy but uses GPU, 'hog' is faster with less accuracy uses cpu
        self.face_encoding_model = face_encoding_model #'large' model has better accuracy but is slower, 'small' model is faster

        self.__face_encodings_pkl_path = './database/known_face_encodings'
        with open(self.__face_encodings_pkl_path, 'rb') as file:
            self.__known_face_encodings = pickle.load(file)

        self.__connection = None
        self.__cursor = None

        self.__mysql_personnel_table = None
        self.__mysql_attendance_table = None
        self.__mysql_unknown_table = None

        self.__time_between_entries = 10 # in seconds


    def get_current_date(self) -> str:
        """
        Get the current date and returns a string representing the current date in the format 'YYYY-MM-DD'.
        """
        current_date = datetime.date.today()
        return str(current_date)
    

    def get_current_time(self) -> str:
        '''Gets the current timestamp with seconds precision, converts it to string, and returns it'''
        return datetime.datetime.now().strftime('%H:%M:%S')
    

    def connect_mysql_database(self, server: str, username: str, password : str, database: str, personnel_table_name: str, attendance_table_name: str, mysql_unknown_name_table: str) -> tuple :
        '''Connects to a MySQL server and sets the table names for 3 required tables'''
        try:
            config = {
                "host": server,
                "user": username,
                "password": password,
                "database": database
            }

            self.__connection = mysql.connector.connect(**config)
            self.__cursor = self.__connection.cursor()
            
            self.__mysql_personnel_table = personnel_table_name
            self.__mysql_attendance_table = attendance_table_name
            self.__mysql_unknown_table = mysql_unknown_name_table
        
        except:
            print("Couldn't connect to MySQL database server! Please check your credentials.")
            exit()


    def initialize_attendance_today(self):
        '''Initializes attendance for the current date (only if it is not already initialized) with every individual as absent by default'''

        self.__cursor.execute(f"SELECT MAX(date) FROM {self.__mysql_attendance_table};")
        latest_date = self.__cursor.fetchone()[0]

        if latest_date is None or latest_date != datetime.date.today():
            self.__cursor.execute(f"SELECT student_id, university FROM {self.__mysql_personnel_table};")
            personnel_data = self.__cursor.fetchall() # Fetch all rows from the "personnel" table

            # Insert data into the "attendance" table
            for row in personnel_data:
                student_id, university = row
                self.__cursor.execute(
                    "INSERT INTO attendance (student_id, university, date, attendance, in_time, entry_exit) VALUES (%s, %s, %s, NULL, NULL, NULL)",
                    (student_id, university, self.get_current_date())
                )

            self.__connection.commit()


    def checking_time_interval(self, time_interval_seconds: int, time_str: str):
        try:
            start_time = datetime.datetime.strptime(time_str, '%H:%M:%S')

            current_time = datetime.datetime.now().strftime('%H:%M:%S')
            current_time = datetime.datetime.strptime(current_time, '%H:%M:%S')

            time_difference = (current_time - start_time).total_seconds() # time interval in seconds

            if time_difference >= time_interval_seconds:
                return True
            else:
                return False

        except ValueError:
            # Handle invalid time strings
            print("Invalid time format. Please use 'HH:MM:SS' format.")
            return None



    def make_attendance(self, student_id: int, student_university: str, date, time):
        self.__cursor.execute(f"SELECT entry_exit FROM {self.__mysql_attendance_table} WHERE student_id = %s AND university = %s AND date = %s", (int(student_id), student_university, date))
        entry_exit = self.__cursor.fetchone()[0]

        if entry_exit is None:
            self.__cursor.execute("SET SQL_SAFE_UPDATES = 0;")
            self.__cursor.execute(
                f"UPDATE {self.__mysql_attendance_table} SET attendance = 1, in_time = %s, entry_exit = %s WHERE student_id = %s AND university = %s AND date = %s",
                (time, time, student_id, student_university, date)
            )
            self.__cursor.execute("SET SQL_SAFE_UPDATES = 1;")
            self.__connection.commit()
        else:
            last_entry = entry_exit.split(',')[-1]
            
            if self.checking_time_interval(self.__time_between_entries, last_entry): # Only execute if desired time interval in seconds have passed between last entry and now
                entry_exit = entry_exit + f',{time}'
                self.__cursor.execute("SET SQL_SAFE_UPDATES = 0;")
                self.__cursor.execute(
                    f"UPDATE {self.__mysql_attendance_table} SET attendance = 1, entry_exit = %s WHERE student_id = %s AND university = %s AND date = %s",
                    (entry_exit, student_id, student_university, date)
                )
                self.__cursor.execute("SET SQL_SAFE_UPDATES = 1;")
                self.__connection.commit()


    def compare_faces(self, known: dict, unknown, tolerance):
        for student_identity, face_encoding in known.items():
            if True in face_recognition.api.compare_faces(face_encoding, unknown, tolerance=tolerance):
                return student_identity
        return None
    

    def start_session(self, camera_index=0, show_preview=True, scale_frame=0.77, desired_fps=2, tolerance=0.45):

        try:

            cap = cv2.VideoCapture(camera_index)
            frame_delay = int(1000 / desired_fps)  # Delay in milliseconds between frames based on the desired FPS
            
            while True:
                ret, frame = cap.read()

                small_frame = cv2.resize(frame, (0, 0), fx=scale_frame, fy=scale_frame) # Resize the frame for faster processing
                rgb_frame = small_frame[:, :, ::-1] # Convert the frame from BGR to RGB

                face_locations = face_recognition.face_locations(rgb_frame, model=self.face_location_model) # Find face locations and face encodings in the frame
                unknown_face_encodings = face_recognition.face_encodings(rgb_frame, face_locations) # Generate encodings of every faces in the frame in a list
                number_of_faces_detected = len(unknown_face_encodings)

                self.__identified_student_ids = []

                if number_of_faces_detected != 0: # Send data only if one or more person is detected
                    date = self.get_current_date()
                    time = self.get_current_time()

                    print(f'[FACE DETECTION] {number_of_faces_detected} faces detected at {time}.')

                    for unknown_face in unknown_face_encodings:
                        id = self.compare_faces(self.__known_face_encodings, unknown_face, tolerance=tolerance)
                        
                        if id is not None:

                            student_id, student_uni, first_name, last_name = id.split('-')

                            # Sending to mysql database
                            self.make_attendance(
                                student_id = student_id,
                                student_university = student_uni,
                                date = date,
                                time = time
                            )

                        else:
                            id = 'Unknown'
                        
                        self.__identified_student_ids.append(id)

                if show_preview == True: 

                    # Draw rectangles around detected faces and display names on 'the scaled frame'
                    for (top, right, bottom, left), identity in zip(face_locations, self.__identified_student_ids):
                        cv2.rectangle(small_frame, (left, top), (right, bottom), (0, 255, 0), 2)
                        cv2.putText(small_frame, identity, (left, bottom + 20), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 255, 0), 1)
                    
                    cv2.imshow('Face Detection (Scaled Frames)', small_frame) # Display the frame with face rectangles
                    
                    if cv2.waitKey(frame_delay) & 0xFF == ord('q'): # Break the loop if 'q' key is pressed
                        break

        except KeyboardInterrupt:
            pass  # Handle keyboard interrupt (e.g., for clean exit)

        finally:
            cap.release()  # Release the camera
            cv2.destroyAllWindows()  # Close OpenCV windows
            self.__connection.close()
            self.__cursor.close()


def main(server, username, password, database, personnel_table_name, attendance_table_name, mysql_unknown_name_table):
    
    session = Session(
        face_location_model='hog', #'cnn' has better accuracy but uses GPU, 'hog' is faster with less accuracy uses cpu
        face_encoding_model = 'small' #'large' model has better accuracy but is slower, 'small' model is faster
    )

    # Connection to MySQL database server
    session.connect_mysql_database(
        server = server,
        username = username,
        password = password,
        database = database,
        personnel_table_name = personnel_table_name, 
        attendance_table_name = attendance_table_name, 
        mysql_unknown_name_table = mysql_unknown_name_table
    ) 

    # Initializing attendance in the 'attendance' table for the current date (only if it is not already initialized) with every individual in the 'personnel' table as absent by default
    session.initialize_attendance_today()

    session.start_session(
        camera_index=0, 
        show_preview=True, 
        scale_frame=0.77, 
        desired_fps=2, 
        tolerance=0.45
    )

def parse_config_file(path='./config.json'):

    with open(path, 'r') as config_file:
        config_data = json.load(config_file)

    server = config_data['server']
    username = config_data['username']
    password = config_data['password']
    database = config_data['database']
    personnel_table_name = config_data['personnel_table_name']
    attendance_table_name = config_data['attendance_table_name']
    mysql_unknown_name_table = config_data['mysql_unknown_name_table']

    return server, username, password, database, personnel_table_name, attendance_table_name, mysql_unknown_name_table


if __name__ == "__main__":

    server, username, password, database, personnel_table_name, attendance_table_name, mysql_unknown_name_table = parse_config_file(path='./config.json')

    main(server, username, password, database, personnel_table_name, attendance_table_name, mysql_unknown_name_table)

[FACE DETECTION] 1 faces detected at 11:36:23.
[FACE DETECTION] 1 faces detected at 11:36:24.
[FACE DETECTION] 1 faces detected at 11:36:26.


In [5]:
face_encodings_pkl_path = './database/known_face_encodings'
with open(face_encodings_pkl_path, 'rb') as file:
    known_face_encodings = pickle.load(file)

known_face_encodings

{'23140736-BCU-ABHASH-RAI': [array([-0.10477734,  0.11417142,  0.05626868,  0.02372511, -0.05847149,
         -0.08704451, -0.11764866, -0.13371034,  0.14709219, -0.09545046,
          0.21196444, -0.07629592, -0.13614778, -0.12347703, -0.01051561,
          0.14970537, -0.19574364, -0.11813708,  0.01023443, -0.06964877,
          0.07544133,  0.00342254,  0.10764163,  0.05608615, -0.15346003,
         -0.40574694, -0.11035254, -0.16486599,  0.02017478, -0.10394485,
         -0.03630624,  0.05989208, -0.14051972, -0.035864  ,  0.00374425,
          0.03960703,  0.0078863 , -0.00218906,  0.1733671 ,  0.06704508,
         -0.19091269,  0.00750373, -0.00618732,  0.33312091,  0.16215658,
          0.0914206 ,  0.00385229, -0.09713081,  0.13492897, -0.12723781,
          0.12913361,  0.12034199,  0.07904803,  0.0275926 ,  0.09950722,
         -0.08649627,  0.0578036 ,  0.06717706, -0.2049017 ,  0.08225445,
          0.10638909, -0.04197368, -0.03394772, -0.03481641,  0.24720539,
          0

In [8]:

get_current_date()

'2023-09-14'